# The PolInSAR Course - June 26, 2024
# SAR Tomography (TomoSAR)
# Part 1: TomoSAR configurations

* Simulated distribution of scatterers
* Path: '/projects/data/05-tomosar/'
    * volume scatterers: pos_scatters_layer1.npy
    * ground scatterers: pos_scatters_ground.npy

Objective:
- Evaluate TomoSAR acquisition configurations starting from a distribution of simulated elementary scatterers. 

Tips:
- 3 functions: one for simulating the SAR data from the distribution of scatterers (already provided), one implementing the TomoSAR reflectivity reconstruction (to be written), and one implementing the calculation of the TomoSAR point-spread function (to be written).

In [ ]:
# import useful libraries, functions, and modules
# To download the data for tomosar
from pysarpro import io, data
from pysarpro.io import rrat

#data.download_all(directory="/projects", pattern=r'^data/tomosar')

import sys
sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
#from ste_io import *
from tqdm import tqdm
import math as ma

%matplotlib widget

In [ ]:
def simulate_data(l, R0, d_orth, pos, dimx, dimy) :
    
    # Simulated a TomoSAR set of images from a distribution of elementary scatterers.
    # Inputs :
    # R0         - range, assumed constant (m)
    # l          - wavelength (m)
    # d_orth     - TomoSAR orthogonal displacements
    # pos        - x, y, z positions of the N scatterers [3 x N] - z is in (m)
    # dimx, dimy - dimensions of the final scene in (x, y)
    # Outputs :
    # y - TomoSAR images (x, y, d_orth.size)
    # !! Achtung !! sub-optimal implementation, fast enough for small scatterer distributions !!
    # Uses tqdm()
    
    K = d_orth.size
    N = pos.shape[1]
    
    # initialize output
    y = np.zeros((dimx, dimy, K), 'complex64')
    
    # calculate y
    for nn in range(N) :
        y[round(pos[0, nn]), round(pos[1, nn]), :] = y[round(pos[0, nn]), round(pos[1, nn]), :] +\
                    np.exp(-1j * 4*np.pi/l/R0 * d_orth * pos[2, nn])
        
    return y
    

In [ ]:
def TomoSAR_reconstruction(l, R0, d_orth, y, z) : 
    
    # Reconstruct the TomoSAR reflectivity from a data set y
    # Inputs :
    # R0     - range, assumed constant (m)
    # l      - wavelength (m)
    # d_orth - TomoSAR orthogonal displacements
    # y      - TomoSAR images (x, y, d_orth.size)
    # z      - height vector (nz) in (m)
    # Outputs :
    # P - TomoSAR complex reflectivity (x, y, d_orth.size)
    # !! Achtung !! assumes constant d_orth and R0 across the scene !!
    
    #P = np.zeros((y.shape[0], y.shape[1], z.size), 'complex64')
    
    # for every pixel: calculate the reference function at different heights and apply it to the data
    K=d_orth.size
    w =  np.exp  (-1j*4*np.pi/l/R0* np.outer(d_orth,z)) # -> 2D matrix (n_tracks,number of heights)
    #w.T -> 2D matrix (number of heights,n_tracks)
    #y  - >TomoSAR data stack    3D,atrix ( dimx,dimy, n_tracks)
    
    #P -> 3D matrix ( dimx,dimy,number of heights)
    P =  np.einsum (  "ht, xyt ->xyh  ",  np.conj(w.T),  y   )    #x -> refers dimx  #y-> refers dimy  #t-> refers n_tracks h -> refers height axis
    
    return P
    # ... to be completed !
    

In [ ]:
def calculate_PSF(l, R0, d_orth, z) : 
    
    # Calculate the TomoSAR PSF
    # Inputs :
    # R0     - range, assumed constant (m)
    # l      - wavelength (m)
    # d_orth - TomoSAR orthogonal displacements
    # y      - TomoSAR images (x, y, d_orth.size)
    # z      - height vector (nz) in (m)
    # Outputs :
    # P - TomoSAR complex reflectivity (x, y, d_orth.size)
    # Uses tqdm() 
    # !! Achtung !! assumes constant d_orth and R0 across the scene !!
    
    # initialize output
    PSF = np.zeros(z.size, 'float32')
    K=d_orth.size
    PSF = 1/K* np.abs( np.sum( np.exp(1j*4*np.pi/l/R0*np.outer (d_orth,z)),axis=0 ))
    
    # ... to be completed !
    
    return PSF
    


**Input parameters**

In [ ]:
# Path to data
path = '/projects/data/tomosar/'

# range - assumed constant - meters
R0 = 5000.
# wavelength - meters
l = 0.2

# scene dimension, x (arbitrary coordinates, positions on file are in the same system)
dimx = 51
# scene dimension, y (arbitrary coordinates, positions on file are in the same system)
dimy = 51

# looks
looksx = 9
looksy = 9

# tomographic height axis
n = np.linspace(-20, 60, 200)


**Load data and visualize**

In [ ]:
# Load data

pos_layer1 = np.load(path + 'pos_scatters_layer1.npy')
pos_ground = np.load(path + 'pos_scatters_ground.npy')

# visualize 3d

fig = plt.figure( figsize = (10, 10) )
ax = plt.axes(projection = '3d')
ax.scatter(pos_layer1[0, :], pos_layer1[1, :], pos_layer1[2, :], color = [0.25,0.75,0.25], s = 3)
ax.scatter(pos_ground[0, :], pos_ground[1, :], pos_ground[2, :], color = [0.75,0.5,0.25], s = 3)
plt.title('3D distribution of points')
ax.set_zlabel('Height (m)')
ax.view_init(9, -57)

# concatenate vectors for simulation
pos = np.concatenate([pos_layer1, pos_ground], axis = 1)


**Case 1: "ideal" acquisition**

- Uniform displacements
- Height ambiguity-free interval = 80 m 
- Height resolution = 5 m

In [ ]:
dn = 5 #resolution
Hfree = 80 #ambiguity

#tomographic aperture
Lx=l/2*R0/dn

#minimum displacement
d=l/2*R0/Hfree

#number of tracks
K=ma.ceil(Lx/d+1)
print(K)

d_orth= np.linspace(0,K-1,K)*d
print(d_orth)

y=simulate_data(l, R0, d_orth, pos, dimx, dimy) 

im_filt=filters.uniform_filter(np.abs(y[:,:,7])**2,[looksx,looksy])
plt.figure()
plt.imshow(im_filt)




fig,grid=plt.subplots(5,4)

for ii in range(K):
    im_filt=filters.uniform_filter((y[:,:,0])*(y[:,:,ii]),[looksx,looksy])
    im_filt=np.abs(im_filt)
    row,column=np.unravel_index(ii,[5,4])
    grid[row,column].imshow(im_filt,cmap="jet")

# --- calculate and plot PSF
z=np.linspace(-20,220,240)


# --- calculate and plot PSF

# ... use the function !
psf=calculate_PSF(l, R0, d_orth, z)
# display
plt.figure( figsize = (7, 5) )
plt.plot(z, psf, lw = 2)
plt.grid()
plt.xlabel('Height (m)')
plt.title('PSF')


# # ... use the function !

P=TomoSAR_reconstruction(l, R0, d_orth, y, z) 

# --- reconstruct reflectivity

# ... use the function and apply a range - azimuth on the intensities
for nn in range(z.size) :
    P[:, :, nn] = np.sqrt(filters.uniform_filter(abs(P[:,:,nn])**2, (looksy, looksx)))

print(P.shape)
print(z.shape)
# display a profile
plt.figure(figsize = (7, 5))
plt.imshow( np.flipud(np.abs(np.transpose(P[:, 15, :]))), cmap = 'jet', \
           aspect = 'auto', extent = [0, dimx-1, np.min(z), np.max(z)])
plt.xlabel('Horizontal direction (pixels)')
plt.ylabel('Height (m)')

 


**Case 2: "ideal" acquisition**

- Uniform displacements
- 13 acquisitions
- Height resolution = 5 m

... and what happens with 7 acquisitions?

In [ ]:
dn = 5.
K  = 7

#tomographic aperture
Lx=l/2*R0/dn

#number of tracks
d=Lx/(K-1)
d_orth= np.linspace(0,K-1,K)*d

print(d_orth,"displacement vector")


y=simulate_data(l, R0, d_orth, pos, dimx, dimy) 

# im_filt=filters.uniform_filter(np.abs(y[:,:,7])**2,[looksx,looksy])
# plt.figure()
# plt.imshow(im_filt)




# fig,grid=plt.subplots(5,4)

# for ii in range(K):
#     im_filt=filters.uniform_filter((y[:,:,0])*(y[:,:,ii]),[looksx,looksy])
#     im_filt=np.abs(im_filt)
#     row,column=np.unravel_index(ii,[5,4])
#     grid[row,column].imshow(im_filt,cmap="jet")

# --- calculate and plot PSF
z=np.linspace(-20,220,240)


# --- calculate and plot PSF

# ... use the function !
psf=calculate_PSF(l, R0, d_orth, z)
# display
plt.figure( figsize = (7, 5) )
plt.plot(z, psf, lw = 2)
plt.grid()
plt.xlabel('Height (m)')
plt.title('PSF')


# # ... use the function !

P=TomoSAR_reconstruction(l, R0, d_orth, y, z) 

# --- reconstruct reflectivity

# ... use the function and apply a range - azimuth on the intensities
for nn in range(z.size) :
    P[:, :, nn] = np.sqrt(filters.uniform_filter(abs(P[:,:,nn])**2, (looksy, looksx)))

print(P.shape)
print(z.shape)
# display a profile
plt.figure(figsize = (7, 5))
plt.imshow( np.flipud(np.abs(np.transpose(P[:, 15, :]))), cmap = 'jet', \
           aspect = 'auto', extent = [0, dimx-1, np.min(z), np.max(z)])
plt.xlabel('Horizontal direction (pixels)')
plt.ylabel('Height (m)')

 




**Case 3: reduced acquisition**

- Uniform displacements
- 7 acquisitions
- Height ambiguity-free interval = 80 m 


In [ ]:
Hfree = 80.
K  = 7 

# minimum displacement
d = l / 2 / Hfree * R0
# tomographic aperture
Lx = (K-1) * d

# displacements
d_orth = np.linspace(0, K-1, K) * d

# --- calculate and plot PSF

psf=calculate_PSF(l, R0, d_orth, z)
# display
plt.figure( figsize = (7, 5) )
plt.plot(z, psf, lw = 2)
plt.grid()
plt.xlabel('Height (m)')
plt.title('PSF')


# # ... use the function !

P=TomoSAR_reconstruction(l, R0, d_orth, y, z) 

# --- reconstruct reflectivity

# ... use the function and apply a range - azimuth on the intensities
for nn in range(z.size) :
    P[:, :, nn] = np.sqrt(filters.uniform_filter(abs(P[:,:,nn])**2, (looksy, looksx)))

print(P.shape)
print(z.shape)
# display a profile
plt.figure(figsize = (7, 5))
plt.imshow( np.flipud(np.abs(np.transpose(P[:, 15, :]))), cmap = 'jet', \
           aspect = 'auto', extent = [0, dimx-1, np.min(z), np.max(z)])
plt.xlabel('Horizontal direction (pixels)')
plt.ylabel('Height (m)')

**Case 4: reduced and non-uniform acquisition**

- Non-Uniform displacements
- 7 acquisitions
- Height ambiguity-free interval = 80 m 
- Height resolution = 5 m 

... and what happens if we accept a resolution of 7.5?

In [ ]:
Hfree = 80.
K  = 7
dn = 5
# minimum displacement
d = l / 2 / Hfree * R0

#tomographic aperture
Lx=l/2*R0/dn

d_orth = np.asarray([ 0, d, 2*d, 5*d, 8*d, 11*d,   Lx  ])
print(d_orth,"displacement vector")



psf=calculate_PSF(l, R0, d_orth, z)
# display
plt.figure( figsize = (7, 5) )
plt.plot(z, psf, lw = 2)
plt.grid()
plt.xlabel('Height (m)')
plt.title('PSF')


# # ... use the function !

P=TomoSAR_reconstruction(l, R0, d_orth, y, z) 

# --- reconstruct reflectivity

# ... use the function and apply a range - azimuth on the intensities
for nn in range(z.size) :
    P[:, :, nn] = np.sqrt(filters.uniform_filter(abs(P[:,:,nn])**2, (looksy, looksx)))

print(P.shape)
print(z.shape)
# display a profile
plt.figure(figsize = (7, 5))
plt.imshow( np.flipud(np.abs(np.transpose(P[:, 15, :]))), cmap = 'jet', \
           aspect = 'auto', extent = [0, dimx-1, np.min(z), np.max(z)])
plt.xlabel('Horizontal direction (pixels)')
plt.ylabel('Height (m)')
